In [1]:
%env OPENAI_API_KEY = api_key

env: OPENAI_API_KEY=api_key


In [2]:
# pip install --upgrade openai

In [3]:
import glob
import json
import os
import re
import random
import time
import torch
from transformers import RobertaModel, RobertaTokenizer
import numpy as np

from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

/home/azmain/.conda/envs/azmain_py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-16 09:50:17.991895: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 09:50:18.087211: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-16 09:50:18.391311: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open share

In [4]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
cd '/home/azmain/alljavajsons'

/home/azmain/alljavajsons


In [6]:
inputFiles = []
correctOutputFiles = []
    
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

for file in sorted(glob.glob("*.java.json"), key=numericalSort):
    inputFiles.append(file)

for file in sorted(glob.glob("*.benchmark_log.json"), key=numericalSort):
    correctOutputFiles.append(file)

print(inputFiles)
print('\n\n\n')
print(correctOutputFiles)

['Android01.java.json', 'Android02.java.json', 'Android03.java.json', 'Android04.java.json', 'Android05.java.json', 'Android06.java.json', 'Android07.java.json', 'Android08.java.json', 'Android09.java.json', 'Android10.java.json', 'Android11.java.json', 'Android12.java.json', 'Android13.java.json', 'Android14.java.json', 'Android15.java.json', 'Android16.java.json', 'Android17.java.json', 'Android18.java.json', 'Android19.java.json', 'Android20.java.json', 'Android21.java.json', 'Android22.java.json', 'Android23.java.json', 'Android24.java.json', 'Android25.java.json', 'Android26.java.json', 'Android27.java.json', 'Android28.java.json', 'Android29.java.json', 'Android30.java.json', 'Android31.java.json', 'Android32.java.json', 'Android33.java.json', 'Android34.java.json', 'Android35.java.json', 'Android36.java.json', 'Android37.java.json', 'Android38.java.json', 'Android39.java.json', 'Android40.java.json', 'Android41.java.json', 'Android42.java.json', 'Android43.java.json', 'Android44

In [7]:
def get_codes(inputFiles):
    codes = []
    for code in inputFiles:
        codes.append(str(json.load(open(code))['originalContent']))

    return codes

In [8]:
def get_correct_outputs(correctOutputFiles):
    correct_outputs = []
    for output in correctOutputFiles:
        correct_output_list = json.load(open(output))['total_imports']
        correct_output_list = ["import "+i+";" for i in correct_output_list]
        correct_outputs.append(correct_output_list)
    
    for import_lines in correct_outputs:
        if "import gen.R;" in import_lines:
            import_lines.remove("import gen.R;")

    return correct_outputs

In [9]:
def get_dataset(codes, correct_outputs):
    dataset = {
        "codes": codes,
        "correct_outputs": correct_outputs
    }
    return dataset

In [10]:
def get_test_examples_and_y_true(dataset):
    sample_list = []

    for i in range(0, len(dataset["codes"])):
        sample_list.append(dict(codes=dataset["codes"][i], correct_outputs=dataset["correct_outputs"][i]))
    
    # print(sample_list)
    
    test_examples = [(example["codes"], example["correct_outputs"]) for example in sample_list]
    y_true = [correct_outputs for _, correct_outputs in test_examples]
    
    return test_examples, y_true

### All Android Classes

In [11]:
print("Total Android Codes: {}\n".format(len(inputFiles[:50])))
print(inputFiles[:50])

android_codes = get_codes(inputFiles[:50])
# print(android_codes)

android_correct_outputs = get_correct_outputs(correctOutputFiles[:50])
# print(android_correct_outputs)

android_dataset = get_dataset(android_codes, android_correct_outputs)
# print(android_dataset)

android_test_examples, android_y_true = get_test_examples_and_y_true(android_dataset)
# print(android_test_examples)

Total Android Codes: 50

['Android01.java.json', 'Android02.java.json', 'Android03.java.json', 'Android04.java.json', 'Android05.java.json', 'Android06.java.json', 'Android07.java.json', 'Android08.java.json', 'Android09.java.json', 'Android10.java.json', 'Android11.java.json', 'Android12.java.json', 'Android13.java.json', 'Android14.java.json', 'Android15.java.json', 'Android16.java.json', 'Android17.java.json', 'Android18.java.json', 'Android19.java.json', 'Android20.java.json', 'Android21.java.json', 'Android22.java.json', 'Android23.java.json', 'Android24.java.json', 'Android25.java.json', 'Android26.java.json', 'Android27.java.json', 'Android28.java.json', 'Android29.java.json', 'Android30.java.json', 'Android31.java.json', 'Android32.java.json', 'Android33.java.json', 'Android34.java.json', 'Android35.java.json', 'Android36.java.json', 'Android37.java.json', 'Android38.java.json', 'Android39.java.json', 'Android40.java.json', 'Android41.java.json', 'Android42.java.json', 'Android

### All JDK Classes

In [12]:
print("Total JDK Codes: {}\n".format(len(inputFiles[50:73])))
print(inputFiles[50:73])

jdk_codes = get_codes(inputFiles[50:73])
# print(jdk_codes)

jdk_correct_outputs = get_correct_outputs(correctOutputFiles[50:73])
# print(jdk_correct_outputs)

jdk_dataset = get_dataset(jdk_codes, jdk_correct_outputs)
# print(jdk_dataset)

jdk_test_examples, jdk_y_true = get_test_examples_and_y_true(jdk_dataset)
# print(jdk_test_examples)

Total JDK Codes: 23

['Class_1.java.json', 'Class_2.java.json', 'Class_3.java.json', 'Class_4.java.json', 'Class_5.java.json', 'Class_6.java.json', 'Class_7.java.json', 'Class_8.java.json', 'Class_9.java.json', 'Class_10.java.json', 'Class_11.java.json', 'Class_12.java.json', 'Class_13.java.json', 'Class_14.java.json', 'Class_15.java.json', 'Class_16.java.json', 'Class_17.java.json', 'Class_18.java.json', 'Class_19.java.json', 'Class_20.java.json', 'Class_21.java.json', 'Class_22.java.json', 'Class_23.java.json']


### All Hibernate Classes

In [13]:
print("Total Hibernate Codes: {}\n".format(len(inputFiles[73:74] + inputFiles[174:224])))
print(inputFiles[73:74] + inputFiles[174:224])

hibernate_codes = get_codes(inputFiles[73:74] + inputFiles[174:224])
# print(hibernate_codes)

hibernate_correct_outputs = get_correct_outputs(correctOutputFiles[73:74] + correctOutputFiles[174:224])
# print(hibernate_correct_outputs)

hibernate_dataset = get_dataset(hibernate_codes, hibernate_correct_outputs)
# print(hibernate_dataset)

hibernate_test_examples, hibernate_y_true = get_test_examples_and_y_true(hibernate_dataset)
# print(hibernate_test_examples)

Total Hibernate Codes: 51

['HibernateUtil.java.json', 'hibernate_class_1.java.json', 'hibernate_class_2.java.json', 'hibernate_class_3.java.json', 'hibernate_class_4.java.json', 'hibernate_class_5.java.json', 'hibernate_class_6.java.json', 'hibernate_class_7.java.json', 'hibernate_class_8.java.json', 'hibernate_class_9.java.json', 'hibernate_class_10.java.json', 'hibernate_class_11.java.json', 'hibernate_class_12.java.json', 'hibernate_class_13.java.json', 'hibernate_class_14.java.json', 'hibernate_class_15.java.json', 'hibernate_class_16.java.json', 'hibernate_class_17.java.json', 'hibernate_class_18.java.json', 'hibernate_class_19.java.json', 'hibernate_class_20.java.json', 'hibernate_class_21.java.json', 'hibernate_class_22.java.json', 'hibernate_class_23.java.json', 'hibernate_class_24.java.json', 'hibernate_class_25.java.json', 'hibernate_class_26.java.json', 'hibernate_class_27.java.json', 'hibernate_class_28.java.json', 'hibernate_class_29.java.json', 'hibernate_class_30.java.j

### All JodaTime Classes

In [14]:
print("Total JodaTime Codes: {}\n".format(len(inputFiles[74:124])))
print(inputFiles[74:124])

jodatime_codes = get_codes(inputFiles[74:124])
# print(jodatime_codes)

jodatime_correct_outputs = get_correct_outputs(correctOutputFiles[74:124])
# print(jodatime_correct_outputs)

jodatime_dataset = get_dataset(jodatime_codes, jodatime_correct_outputs)
# print(jodatime_dataset)

jodatime_test_examples, jodatime_y_true = get_test_examples_and_y_true(jodatime_dataset)
# print(jodatime_test_examples)

Total JodaTime Codes: 50

['JodaTime01.java.json', 'JodaTime02.java.json', 'JodaTime03.java.json', 'JodaTime04.java.json', 'JodaTime05.java.json', 'JodaTime06.java.json', 'JodaTime07.java.json', 'JodaTime08.java.json', 'JodaTime09.java.json', 'JodaTime10.java.json', 'JodaTime11.java.json', 'JodaTime12.java.json', 'JodaTime13.java.json', 'JodaTime14.java.json', 'JodaTime15.java.json', 'JodaTime16.java.json', 'JodaTime17.java.json', 'JodaTime18.java.json', 'JodaTime19.java.json', 'JodaTime20.java.json', 'JodaTime21.java.json', 'JodaTime22.java.json', 'JodaTime23.java.json', 'JodaTime24.java.json', 'JodaTime25.java.json', 'JodaTime26.java.json', 'JodaTime27.java.json', 'JodaTime28.java.json', 'JodaTime29.java.json', 'JodaTime30.java.json', 'JodaTime31.java.json', 'JodaTime32.java.json', 'JodaTime33.java.json', 'JodaTime34.java.json', 'JodaTime35.java.json', 'JodaTime36.java.json', 'JodaTime37.java.json', 'JodaTime38.java.json', 'JodaTime39.java.json', 'JodaTime40.java.json', 'JodaTime41.j

### All GWT Classes

In [15]:
print("Total GWT Codes: {}\n".format(len(inputFiles[124:174])))
print(inputFiles[124:174])

gwt_codes = get_codes(inputFiles[124:174])
# print(gwt_codes)

gwt_correct_outputs = get_correct_outputs(correctOutputFiles[124:174])
# print(gwt_correct_outputs)

gwt_dataset = get_dataset(gwt_codes, gwt_correct_outputs)
# print(gwt_dataset)

gwt_test_examples, gwt_y_true = get_test_examples_and_y_true(gwt_dataset)
# print(gwt_test_examples)

Total GWT Codes: 50

['gwt_class_1.java.json', 'gwt_class_2.java.json', 'gwt_class_3.java.json', 'gwt_class_4.java.json', 'gwt_class_5.java.json', 'gwt_class_6.java.json', 'gwt_class_7.java.json', 'gwt_class_8.java.json', 'gwt_class_9.java.json', 'gwt_class_10.java.json', 'gwt_class_11.java.json', 'gwt_class_12.java.json', 'gwt_class_13.java.json', 'gwt_class_14.java.json', 'gwt_class_15.java.json', 'gwt_class_16.java.json', 'gwt_class_17.java.json', 'gwt_class_18.java.json', 'gwt_class_19.java.json', 'gwt_class_20.java.json', 'gwt_class_21.java.json', 'gwt_class_22.java.json', 'gwt_class_23.java.json', 'gwt_class_24.java.json', 'gwt_class_25.java.json', 'gwt_class_26.java.json', 'gwt_class_27.java.json', 'gwt_class_28.java.json', 'gwt_class_29.java.json', 'gwt_class_30.java.json', 'gwt_class_31.java.json', 'gwt_class_32.java.json', 'gwt_class_33.java.json', 'gwt_class_34.java.json', 'gwt_class_35.java.json', 'gwt_class_36.java.json', 'gwt_class_37.java.json', 'gwt_class_38.java.json',

### All XStream Classes

In [16]:
print("Total XStream Codes: {}\n".format(len(inputFiles[224:268])))
print(inputFiles[224:268])

xstream_codes = get_codes(inputFiles[224:268])
# print(xstream_codes)

xstream_correct_outputs = get_correct_outputs(correctOutputFiles[224:268])
# print(xstream_correct_outputs)

xstream_dataset = get_dataset(xstream_codes, xstream_correct_outputs)
# print(xstream_dataset)

xstream_test_examples, xstream_y_true = get_test_examples_and_y_true(xstream_dataset)
# print(xstream_test_examples)

Total XStream Codes: 44

['xstream_class_1.java.json', 'xstream_class_2.java.json', 'xstream_class_3.java.json', 'xstream_class_4.java.json', 'xstream_class_5.java.json', 'xstream_class_6.java.json', 'xstream_class_7.java.json', 'xstream_class_8.java.json', 'xstream_class_9.java.json', 'xstream_class_10.java.json', 'xstream_class_11.java.json', 'xstream_class_12.java.json', 'xstream_class_13.java.json', 'xstream_class_14.java.json', 'xstream_class_15.java.json', 'xstream_class_16.java.json', 'xstream_class_17.java.json', 'xstream_class_18.java.json', 'xstream_class_19.java.json', 'xstream_class_20.java.json', 'xstream_class_21.java.json', 'xstream_class_22.java.json', 'xstream_class_23.java.json', 'xstream_class_24.java.json', 'xstream_class_25.java.json', 'xstream_class_26.java.json', 'xstream_class_27.java.json', 'xstream_class_28.java.json', 'xstream_class_29.java.json', 'xstream_class_30.java.json', 'xstream_class_31.java.json', 'xstream_class_32.java.json', 'xstream_class_33.java.

# One-shot and Self-Consistency Implementation

In [17]:
MODEL = 'gpt-3.5-turbo'

In [18]:
def pred_process(y_pred, y_true):
    y_pred_processed = []
    y_true_processed = []
    
    for pred, correct_imports in zip(y_pred, y_true):
        max_length = max(len(pred), len(correct_imports))
        correct_preds = list(set(pred).intersection(correct_imports))
#         print('Correct Predictions:', correct_preds)
#         wrong_preds = max_length - len(correct_preds)
#         print('Wrong Predictions:', wrong_preds)

        for i in range(0, max_length):
            if i<len(correct_preds):
                y_pred_processed.append(1)
                y_true_processed.append(1)
            else:
                if i<len(correct_imports):
                    y_pred_processed.append(0)
                    y_true_processed.append(1)
                else:
                    y_pred_processed.append(1)
                    y_true_processed.append(0)
            
    print(y_pred_processed)
    print(y_true_processed)
    print()
    return y_pred_processed, y_true_processed

In [19]:
def eval_performance(y_pred, y_true):
    print(json.dumps({
        "accuracy": accuracy_score(y_pred, y_true),
        "f1": f1_score(y_pred, y_true),
        "precision": precision_score(y_pred, y_true),
        "recall": recall_score(y_pred, y_true)
    }, indent=2))

In [20]:
# # Test

# print(xstream_test_examples[39][1]) # correct imports
# print(xstream_test_examples[39][0]) # code

In [21]:
# # Test

# MODEL = 'gpt-3.5-turbo'

# code_snippet = xstream_test_examples[39][0]
# # prompt=f"Reply with to-the-point answer, with no elaboration. Extract all valid possible fully qualified type long-name from the code below, must exclude name of the class itself:\n\"\"\"{test_code}\"\"\""

# response = openai.ChatCompletion.create(
#     model=MODEL,
#     messages=[
#         {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
# #         {"role": "user", "content": f"Import correct imports:\n\"\"\"\n{test_code}\n\"\"\""},
# #         {"role": "user", "content": f"Your task is to provide a list of the correct imports for a given code snippet, ensuring that no wildcard imports are used. If any necessary imports are missing from the code, please include them in your response. Please note that your response should be specific and accurate:\n\"\"\"\n{code_snippet}\n\"\"\""},
# #         {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that your response should be specific and accurate:\n\"\"\"\n{code_snippet}\n\"\"\""},
#         {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay very close attention, and your response should be specific and accurate. You are prohibited from importing wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""},
        
#     ],
#     temperature=1,
# )

# print(response["choices"][0]["message"]["content"])

# One-shot

In [22]:
def get_oneshot_prediction(example, code):
    for code_snippet, correct_output in example:
        expected_output = '\n'.join(correct_output)
    
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
            {"role": "system", "name":"example_user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""},
            {"role": "system", "name": "example_assistant", "content": f"{expected_output}"},
            {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code}\n\"\"\""},
        ],
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [23]:
def get_all_oneshot_predictions(example, dataset):
    y_pred = []
    for code_snippet, correct_imports in tqdm(dataset):
        if code_snippet in example[0][0]:
            predicted_output = get_oneshot_prediction([example[1]], code_snippet)
        else:
            predicted_output = get_oneshot_prediction([example[0]], code_snippet)
        y_pred.append(re.findall(r"import\s+[\w\., ]+;", predicted_output))
        time.sleep(5)
    return y_pred

In [24]:
def get_oneshot_sample(dataset):
    oneshot_list = []
    
    for i in range(0, len(dataset["codes"])):
        oneshot_list.append(dict(codes=dataset["codes"][i], correct_outputs=dataset["correct_outputs"][i]))
        
    oneshot_example = [(example["codes"], example["correct_outputs"]) for example in oneshot_list]
    oneshot_sample = random.sample(oneshot_example, 2)
    
    # print(oneshot_sample)
    return oneshot_sample

In [25]:
# One-shot Prediction for Android Classes

print("\nOne-shot Prediction for Android Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(android_dataset), android_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", android_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)


One-shot Prediction for Android Classes:



100%|██████████| 50/50 [05:58<00:00,  7.18s/it]


Predicted Import List: [['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.app.Activity;', 'import android.os.Bundle;'], ['import android.graphics.drawable.Drawable;', 'import android.os.Bundle;', 'import android.util.Log;', 'import android.widget.Toast;', 'import com.google.android.maps.ItemizedOverlay;', 'import com.google.android.maps.OverlayItem;', 'import com.google.android.maps.GeoPoint;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.graphics.drawable.Drawable;', 'import android.widget.TabHost;', 'import android.widget.TabHost.TabSpec;'], ['import android.view.Gravity;', 'import android.view.ViewGroup;', 'import android.widget.LinearLayout;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.widget.ArrayAdapter;'], ['import java.io.IOException;', 'import java.net.Inet4Address;', 'import ja

In [27]:
# One-shot Prediction for JDK Classes

print("\nOne-shot Prediction for JDK Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(jdk_dataset), jdk_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jdk_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)


One-shot Prediction for JDK Classes:



100%|██████████| 23/23 [02:49<00:00,  7.37s/it]


Predicted Import List: [['import java.applet.Applet;', 'import java.awt.Color;', 'import java.awt.Container;', 'import java.awt.Dimension;', 'import java.awt.Frame;', 'import java.awt.Graphics;', 'import java.awt.Graphics2D;', 'import java.awt.event.WindowAdapter;', 'import java.awt.event.WindowEvent;', 'import java.awt.image.BufferedImage;', 'import java.io.FileInputStream;', 'import java.io.FileOutputStream;', 'import java.io.IOException;', 'import java.io.ObjectInputStream;', 'import java.io.ObjectOutputStream;'], ['import java.awt.BorderLayout;', 'import java.awt.Color;', 'import java.awt.Dimension;', 'import java.awt.FlowLayout;', 'import java.awt.GradientPaint;', 'import java.awt.Graphics2D;', 'import java.awt.GridBagLayout;', 'import java.awt.GridLayout;', 'import java.awt.event.ActionEvent;', 'import java.awt.event.ActionListener;', 'import java.awt.image.BufferedImage;', 'import javax.swing.JButton;', 'import javax.swing.JCheckBox;', 'import javax.swing.JComboBox;', 'import j

In [28]:
# One-shot Prediction for Hibernate Classes

print("\nOne-shot Prediction for Hibernate Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(hibernate_dataset), hibernate_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", hibernate_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)


One-shot Prediction for Hibernate Classes:



100%|██████████| 51/51 [06:09<00:00,  7.25s/it]


Predicted Import List: [['import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.AnnotationConfiguration;'], ['import org.hibernate.SessionFactory;', 'import org.hibernate.Session;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.AnnotationConfiguration;', 'import hibernate.User;', 'import hibernate.UserDAO;'], ['import java.io.Serializable;', 'import java.util.List;', 'import javax.persistence.CacheConcurrencyStrategy;', 'import javax.persistence.Column;', 'import javax.persistence.Entity;', 'import javax.persistence.Id;', 'import javax.persistence.JoinColumn;', 'import javax.persistence.ManyToOne;', 'import javax.persistence.OneToMany;'], ['import org.hibernate.dialect.Dialect;', 'import org.hibernate.Hibernate;', 'import org.hibernate.dialect.function.SQLFunctionTemplate;', 'import org.hibernate.dialect.function.StandardSQLFunction;', 'import org.hibernate.dialect.function.VarArgsSQLFunction;', 'import java.sql.Types;'], ['import org.hibernate.SessionFacto

In [29]:
# One-shot Prediction for Joda-Time Classes

print("\nOne-shot Prediction for Joda-Time Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(jodatime_dataset), jodatime_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jodatime_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)


One-shot Prediction for Joda-Time Classes:



100%|██████████| 50/50 [05:32<00:00,  6.65s/it]


Predicted Import List: [['import org.joda.time.DateTime;', 'import org.joda.time.DateTimeZone;', 'import org.joda.time.format.DateTimeFormatter;', 'import org.joda.time.format.ISODateTimeFormat;', 'import java.util.TimeZone;'], ['import org.joda.time.DateMidnight;', 'import org.joda.time.DateTime;', 'import org.joda.time.DateTimeZone;'], ['import org.joda.time.Interval;', 'import org.joda.time.PeriodFormatter;', 'import org.joda.time.PeriodFormatterBuilder;'], ['import org.joda.time.Period;', 'import org.joda.time.ReadableInstant;'], ['import org.joda.time.DateTimeZone;', 'import org.joda.time.Chronology;', 'import org.joda.time.DateTime;', 'import org.joda.time.chrono.GJChronology;'], ['import org.joda.time.Duration;', 'import org.joda.time.PeriodType;', 'import org.joda.time.format.PeriodFormatterBuilder;'], ['import java.text.DateFormat;', 'import java.text.ParseException;', 'import java.text.SimpleDateFormat;', 'import java.util.Calendar;', 'import java.util.Date;', 'import org.jo

In [30]:
# One-shot Prediction for GWT Classes

print("\nOne-shot Prediction for GWT Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(gwt_dataset), gwt_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", gwt_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)


One-shot Prediction for GWT Classes:



100%|██████████| 50/50 [09:34<00:00, 11.49s/it]


Predicted Import List: [['import com.google.gwt.event.dom.client.MouseDownEvent;', 'import com.google.gwt.event.dom.client.MouseDownHandler;', 'import com.google.gwt.user.client.ui.AbsolutePanel;', 'import com.google.gwt.user.client.ui.Composite;'], ['import com.google.gwt.core.client.GWT;', 'import com.google.gwt.junit.client.GWTTestCase;', 'import com.google.gwt.user.client.rpc.AsyncCallback;', 'import com.google.gwt.sample.stockwatcher.client.GreetingService;', 'import com.google.gwt.sample.stockwatcher.client.GreetingServiceAsync;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.user.client.Window;', 'import com.google.gwt.user.client.rpc.AsyncCallback;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.TextBox;'], ['import com.google.gwt.user.client.ui.Composite;', 'import com.google.gwt.user.client.ui.VerticalSplitPanel;', 'import com.google.gwt.user.client.ui.HTML;

In [32]:
# One-shot Prediction for XStream Classes

print("\nPrediction for XStream Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(xstream_dataset), xstream_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", xstream_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)


Prediction for XStream Classes:



100%|██████████| 44/44 [05:37<00:00,  7.67s/it]


Predicted Import List: [['import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.DomDriver;', 'import java.io.BufferedReader;', 'import java.io.FileReader;', 'import java.io.IOException;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.json.JettisonMappedXmlDriver;', 'import java.util.ArrayList;', 'import java.util.List;'], ['import java.util.ArrayList;', 'import com.thoughtworks.xstream.XStream;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.DomDriver;', 'import java.io.FileNotFoundException;', 'import java.io.PrintWriter;'], ['import com.thoughtworks.xstream.XStream

# Exracting Example using Semantic Similarity

In [32]:
#

In [25]:
def get_semantic_oneshot_prediction(semantic_example, code):
    time.sleep(5)
    for code_snippet, example_output in semantic_example:
        expected_output = '\n'.join(example_output)
    
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
            {"role": "system", "name":"example_user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""},
            {"role": "system", "name": "example_assistant", "content": f"{expected_output}"},
            {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code}\n\"\"\""},
        ],
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [26]:
def get_all_semantic_oneshot_predictions(semantic_example, dataset):
    y_pred = []
    i = 0
    for code, correct_imports in tqdm(dataset):
        predicted_output = get_semantic_oneshot_prediction([semantic_example[i]], code)
        y_pred.append(re.findall(r"import\s+[\w\., ]+;", predicted_output))
        i = i+1
    return y_pred

In [27]:
def get_semantic_similarity(dataset):
    # Load pre-trained model and tokenizer
    model = RobertaModel.from_pretrained('microsoft/codebert-base')
    tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')

    # List to store code embeddings
    code_embeddings = []

    # Tokenize and encode all the code snippets and store the embeddings
    for code in dataset['codes']:
        inputs = tokenizer(code, return_tensors='pt', truncation=True, max_length=512, padding='max_length')
        with torch.no_grad():
            outputs = model(**inputs)
        embedding = outputs[0][0][0]
        code_embeddings.append(embedding)

    # Convert list to tensor for easier manipulation
    code_embeddings_tensor = torch.stack(code_embeddings)

    # Calculate pairwise cosine similarity
    cos_sim_matrix = torch.nn.functional.cosine_similarity(
        code_embeddings_tensor.unsqueeze(1), code_embeddings_tensor.unsqueeze(0), dim=2
    )

    # Fill the diagonal with large negative numbers to ignore self-comparison
    np.fill_diagonal(cos_sim_matrix.numpy(), float('-inf'))

    # Get the index of the most similar code for each code
    most_similar_indices = cos_sim_matrix.argmax(dim=1)

    # Construct a new list of tuples with similar codes and their own imports
    semantic_example_list = []
    for i, idx in enumerate(most_similar_indices):
        semantic_example_list.append((dataset['codes'][idx.item()], dataset['correct_outputs'][idx.item()]))

    return semantic_example_list

In [28]:
# One-shot Prediction for Android Classes using semantic similarity

print("\nOne-shot Prediction for Android Classes using semantic similarity:\n")
android_semantic_list = get_semantic_similarity(android_dataset)
print(f"Android Semantic List:\n\n{android_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(android_semantic_list, android_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", android_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)


One-shot Prediction for Android Classes using semantic similarity:

Android Semantic List:

[('package androidExamples;\n//ID = 151777\n\npublic class Android04 extends Activity {\n    /** Called when the activity is first created. */\n    @Override\n    public void onCreate(Bundle savedInstanceState) {\n        super.onCreate(savedInstanceState);\n\n        mTextView = new TextView(this);\n\n        if (savedInstanceState == null) {\n            mTextView.setText("Welcome to HelloAndroid!");\n        } else {\n            mTextView.setText("Welcome back.");\n        }\n\n        setContentView(mTextView);\n    }\n\n    private TextView mTextView = null;\n}', ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;']), ('package androidExamples;\n//ID = 151777\n\npublic class Android04 extends Activity {\n    /** Called when the activity is first created. */\n    @Override\n    public void onCreate(Bundle savedInstanceState) {\n        super.onCre

100%|██████████| 50/50 [04:53<00:00,  5.87s/it]


Predicted Import List: [['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.app.Activity;', 'import android.os.Bundle;'], ['import android.graphics.drawable.Drawable;', 'import com.google.android.maps.ItemizedOverlay;', 'import com.google.android.maps.OverlayItem;', 'import com.google.android.maps.GeoPoint;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.graphics.drawable.Drawable;', 'import android.widget.TabHost;', 'import android.widget.TabSpec;'], ['import android.view.ViewGroup;', 'import android.view.Gravity;', 'import android.widget.LinearLayout;', 'import android.view.View;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.widget.ArrayAdapter;'], ['import java.io.IOException;', 'import java.net.Inet4Address;', 'import java.net.InetAddress;', 'import java.net.Socket;', 'import java.net.Un

In [ ]:
# One-shot Prediction for JDK Classes using semantic similarity

print("\nOne-shot Prediction for JDK Classes using semantic similarity:\n")
jdk_semantic_list = get_semantic_similarity(jdk_dataset)
print(f"JDK Semantic List:\n\n{jdk_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(jdk_semantic_list, jdk_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jdk_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)


One-shot Prediction for JDK Classes using semantic similarity:

JDK Semantic List:

[('/*\nDate: 1/19/17\nURL: http://stackoverflow.com/questions/7143287/how-to-best-position-swing-guis\n*/\npackage common_JDK;\nclass WhereToPutTheGui {\n\n    public static void initGui() {\n        for (int ii=1; ii<4; ii++) {\n            JFrame f = new JFrame("Frame " + ii);\n            f.setDefaultCloseOperation(JFrame.DISPOSE_ON_CLOSE);\n            String s =\n                "os.name: " + System.getProperty("os.name") +\n                "\\nos.version: " + System.getProperty("os.version");\n            f.add(new JTextArea(s,3,28));  // suggest a size\n            f.pack();\n            // Let the OS handle the positioning!\n            f.setLocationByPlatform(true);\n            f.setVisible(true);\n        }\n    }\n\n    public static void main(String[] args) {\n        SwingUtilities.invokeLater( new Runnable() {\n            public void run() {\n                try {\n                    U

 78%|███████▊  | 18/23 [01:04<00:15,  3.01s/it]

In [ ]:
# One-shot Prediction for Hibernate Classes using semantic similarity

print("\nOne-shot Prediction for Hibernate Classes using semantic similarity:\n")
hibernate_semantic_list = get_semantic_similarity(hibernate_dataset)
print(f"Hibernate Semantic List:\n\n{hibernate_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(hibernate_semantic_list, hibernate_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", hibernate_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for Joda-Time Classes using semantic similarity

print("\nOne-shot Prediction for Joda-Time Classes using semantic similarity:\n")
jodatime_semantic_list = get_semantic_similarity(jodatime_dataset)
print(f"Joda-Time Semantic List:\n\n{jodatime_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(jodatime_semantic_list, jodatime_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jodatime_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for GWT Classes using semantic similarity

print("\nOne-shot Prediction for GWT Classes using semantic similarity:\n")
gwt_semantic_list = get_semantic_similarity(gwt_dataset)
print(f"GWT Semantic List:\n\n{gwt_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(gwt_semantic_list, gwt_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", gwt_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for XStream Classes using semantic similarity

print("\nOne-shot Prediction for XStream Classes using semantic similarity:\n")
xstream_semantic_list = get_semantic_similarity(xstream_dataset)
print(f"XStream Semantic List:\n\n{xstream_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(xstream_semantic_list, xstream_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", xstream_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)

## Self-Consistency with Modified Parameters (The consistent one from 10 predictions for each input)

In [33]:
from collections import Counter

def get_consistent_item(k_items_list, code_snippet, one_shot_example):
    
    # Count the frequency of each set of sublists
    counts = Counter(frozenset(sublist) for sublist in k_items_list)

    # Find the maximum frequency count
    max_count = max(counts.values())

    # Find all sets of sublists with the maximum frequency count
    most_common_sublists_set = {frozenset(sublist) for sublist in counts if counts[sublist] == max_count}

    # Convert the sets of sublists back to lists
    most_common_sublists = [list(sublist) for sublist in most_common_sublists_set]

    # Find the number of sublists that have the highest frequency count
    max_consistency_count = len(most_common_sublists)
    
    max_check_time = 5
    i = 0
    while max_consistency_count!=1:
        if i > max_check_time:
            return []
        time.sleep(2)
        new_predicted_output = re.findall(r"import\s+[\w\., ]+;", get_modified_temp_prediction(one_shot_example, code_snippet))
        k_items_list.append(new_predicted_output)
        counts = Counter(frozenset(sublist) for sublist in k_items_list)
        max_count = max(counts.values())
        most_common_sublists_set = {frozenset(sublist) for sublist in counts if counts[sublist] == max_count}
        most_common_sublists = [list(sublist) for sublist in most_common_sublists_set]
        max_consistency_count = len(most_common_sublists)
        i+=1

    consistent_item = most_common_sublists[0]
    
    return consistent_item

In [34]:
def get_modified_temp_prediction(example, code_snippet):
    for example_code, correct_output in example:
        expected_output = '\n'.join(correct_output)
        
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
            {"role": "system", "name":"example_user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{example_code}\n\"\"\""},
            {"role": "system", "name": "example_assistant", "content": f"{expected_output}"},
            {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""},
        ],
        temperature=1,
        top_p=1,
    )
    return response["choices"][0]["message"]["content"]

In [35]:
def get_all_predictions(example, dataset):
    y_pred_list = []
    one_shot_example = []
    for code_snippet, correct_imports in tqdm(dataset):
        if code_snippet in example[0][0]:
            one_shot_example = [example[1]]
        else:
            one_shot_example = [example[0]]
        k_output_sample = []
        k = 10
        for i in range(0, k):
            time.sleep(2)
            predicted_output = re.findall(r"import\s+[\w\., ]+;", get_modified_temp_prediction(one_shot_example, code_snippet))
            k_output_sample.append(predicted_output)
        y_pred_list.append(get_consistent_item(k_output_sample, code_snippet, one_shot_example))
    return y_pred_list

In [ ]:
# Prediction with One-shot Self-Consistency and Modified Parameters for Android Classes

print("\nPrediction with One-shot Self-Consistency and Modified Parameters for Android Classes:\n")
y_pred = get_all_predictions(get_oneshot_sample(android_dataset), android_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", android_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with One-shot Self-Consistency and Modified Parameters for JDK Classes

print("\nPrediction with One-shot Self-Consistency and Modified Parameters for JDK Classes:\n")
y_pred = get_all_predictions(get_oneshot_sample(jdk_dataset), jdk_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jdk_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with One-shot Self-Consistency and Modified Parameters for Hibernate Classes

print("\nPrediction with One-shot Self-Consistency and Modified Parameters for Hibernate Classes:\n")
y_pred = get_all_predictions(get_oneshot_sample(hibernate_dataset), hibernate_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", hibernate_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with One-shot Self-Consistency and Modified Parameters for Joda-Time Classes

print("\nPrediction with One-shot Self-Consistency and Modified Parameters for Joda-Time Classes:\n")
y_pred = get_all_predictions(get_oneshot_sample(jodatime_dataset), jodatime_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jodatime_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with One-shot Self-Consistency and Modified Parameters for GWT Classes

print("\nPrediction with One-shot Self-Consistency and Modified Parameters for GWT Classes:\n")
y_pred = get_all_predictions(get_oneshot_sample(gwt_dataset), gwt_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", gwt_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with One-shot Self-Consistency and Modified Parameters for XStream Classes

print("\nPrediction with One-shot Self-Consistency and Modified Parameters for XStream Classes:\n")
y_pred = get_all_predictions(get_oneshot_sample(xstream_dataset), xstream_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", xstream_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)